# Olist E-Commerce Data

## **Setting up Spark Environment**

1\. Deploy a spark cluster (Dataproc, EMR, HDInsight, On-Premis)

2\. Download the data from Kaggle website [Link](https://www.kaggle.com/api/v1/datasets/download/olistbr/brazilian-ecommerce)

3\. Bash command to direct download to HDFS 
- Make sure to create a proper directory to put downloaded and to extract files into
- I created olist folder to download file
```
curl -L -o ~/olist/brazilian-ecommerce.zip\
  https://www.kaggle.com/api/v1/datasets/download/olistbr/brazilian-ecommerce
```
- I created data folder inside olist to extract files

```
unzip brazilian-ecommerce.zip -d ~/olist/data/
```

4\. Store Data in HDFS worker nodes
- I create folder to distribute files from hadoop master node to worker nodes
```
!hadoop fs -put ~/olist/data/*.csv /data/olist/
```
    
5\. Use Pyspark to intract with Data



# Data Ingestion

## **Downloaded and extracted data in hadoop master cluster**

In [21]:
ls /home/sekar_dhana8644/olist/

brazilian-ecommerce.zip  data/


In [20]:
ls /home/sekar_dhana8644/olist/data/

olist_customers_dataset.csv       olist_orders_dataset.csv
olist_geolocation_dataset.csv     olist_products_dataset.csv
olist_order_items_dataset.csv     olist_sellers_dataset.csv
olist_order_payments_dataset.csv  product_category_name_translation.csv
olist_order_reviews_dataset.csv


## **Moved data from Hadoop master cluster to worker nodes**

In [19]:
!hadoop fs -ls -h /data/olist/

Found 9 items
-rw-r--r--   2 sekar_dhana8644 hadoop      8.6 M 2025-08-30 19:17 /data/olist/olist_customers_dataset.csv
-rw-r--r--   2 sekar_dhana8644 hadoop     58.4 M 2025-08-30 19:17 /data/olist/olist_geolocation_dataset.csv
-rw-r--r--   2 sekar_dhana8644 hadoop     14.7 M 2025-08-30 19:17 /data/olist/olist_order_items_dataset.csv
-rw-r--r--   2 sekar_dhana8644 hadoop      5.5 M 2025-08-30 19:17 /data/olist/olist_order_payments_dataset.csv
-rw-r--r--   2 sekar_dhana8644 hadoop     13.8 M 2025-08-30 19:17 /data/olist/olist_order_reviews_dataset.csv
-rw-r--r--   2 sekar_dhana8644 hadoop     16.8 M 2025-08-30 19:17 /data/olist/olist_orders_dataset.csv
-rw-r--r--   2 sekar_dhana8644 hadoop      2.3 M 2025-08-30 19:17 /data/olist/olist_products_dataset.csv
-rw-r--r--   2 sekar_dhana8644 hadoop    170.6 K 2025-08-30 19:17 /data/olist/olist_sellers_dataset.csv
-rw-r--r--   2 sekar_dhana8644 hadoop      2.6 K 2025-08-30 19:17 /data/olist/product_category_name_translation.csv


# Data Exploration

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('OlistDataset').getOrCreate()

spark

25/08/31 18:17:50 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [1]:
hdfs_path = '/data/olist/'

In [2]:
customer_df = spark.read.csv(hdfs_path + 'olist_customers_dataset.csv',header=True, inferSchema=True)

In [3]:
customer_df.show(5)

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
+--------------------+--------------------+------------------------+--------------------+--------------+
only showing top 5 rows



In [4]:
geolocation_df = spark.read.csv(hdfs_path + 'olist_geolocation_dataset.csv',header=True, inferSchema=True)
order_items_df = spark.read.csv(hdfs_path + 'olist_order_items_dataset.csv',header=True, inferSchema=True)
order_payments_df = spark.read.csv(hdfs_path + 'olist_order_payments_dataset.csv',header=True, inferSchema=True)
order_reviews_df = spark.read.csv(hdfs_path + 'olist_order_reviews_dataset.csv',header=True, inferSchema=True)
orders_df = spark.read.csv(hdfs_path + 'olist_orders_dataset.csv',header=True, inferSchema=True)
products_df = spark.read.csv(hdfs_path + 'olist_products_dataset.csv',header=True, inferSchema=True)
sellers_df = spark.read.csv(hdfs_path + 'olist_sellers_dataset.csv',header=True, inferSchema=True)
translation_df = spark.read.csv(hdfs_path + 'product_category_name_translation.csv',header=True, inferSchema=True)

In [5]:
customer_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [6]:
orders_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



In [7]:
# Data Leakage or Drop

print(f'Customers df: {customer_df.count()} rows')
print(f'Orders df: {orders_df.count()} rows')
print(f'Order Items df: {order_items_df.count()} rows')
print(f'Order Payments df: {order_payments_df.count()} rows')
print(f'Order Reviews df: {order_reviews_df.count()} rows')
print(f'Products df: {products_df.count()} rows')
print(f'Sellers df: {sellers_df.count()} rows')
print(f'Product Category Name Translation df: {translation_df.count()} rows')

Customers df: 99441 rows
Orders df: 99441 rows
Order Items df: 112650 rows
Order Payments df: 103886 rows
Order Reviews df: 104162 rows
Products df: 32951 rows
Sellers df: 3095 rows
Product Category Name Translation df: 71 rows


In [8]:
# Null Values

from pyspark.sql.functions import *

customer_df.select([count(when(col(c).isNull(),1)).alias(c) for c in customer_df.columns]).show()

+-----------+------------------+------------------------+-------------+--------------+
|customer_id|customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|
+-----------+------------------+------------------------+-------------+--------------+
|          0|                 0|                       0|            0|             0|
+-----------+------------------+------------------------+-------------+--------------+



In [9]:
# Duplicate Values

customer_df.groupBy('customer_id').count().filter('count>1').show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
+-----------+-----+



In [10]:
# Customer Distribution by State

customer_df.groupBy('customer_state').count().orderBy(col('count').desc()).show()

+--------------+-----+
|customer_state|count|
+--------------+-----+
|            SP|41746|
|            RJ|12852|
|            MG|11635|
|            RS| 5466|
|            PR| 5045|
|            SC| 3637|
|            BA| 3380|
|            DF| 2140|
|            ES| 2033|
|            GO| 2020|
|            PE| 1652|
|            CE| 1336|
|            PA|  975|
|            MT|  907|
|            MA|  747|
|            MS|  715|
|            PB|  536|
|            PI|  495|
|            RN|  485|
|            AL|  413|
+--------------+-----+
only showing top 20 rows



In [11]:
# Customer Distribution by City

customer_df.groupBy('customer_city').count().orderBy('count', ascending=False).show()

+--------------------+-----+
|       customer_city|count|
+--------------------+-----+
|           sao paulo|15540|
|      rio de janeiro| 6882|
|      belo horizonte| 2773|
|            brasilia| 2131|
|            curitiba| 1521|
|            campinas| 1444|
|        porto alegre| 1379|
|            salvador| 1245|
|           guarulhos| 1189|
|sao bernardo do c...|  938|
|             niteroi|  849|
|         santo andre|  797|
|              osasco|  746|
|              santos|  713|
|             goiania|  692|
| sao jose dos campos|  691|
|           fortaleza|  654|
|            sorocaba|  633|
|              recife|  613|
|       florianopolis|  570|
+--------------------+-----+
only showing top 20 rows



In [12]:
orders_df.columns

['order_id',
 'customer_id',
 'order_status',
 'order_purchase_timestamp',
 'order_approved_at',
 'order_delivered_carrier_date',
 'order_delivered_customer_date',
 'order_estimated_delivery_date']

In [13]:
# Order Distribution by Order Status

orders_df.groupBy('order_status').count().orderBy('count', ascending=False).show()

+------------+-----+
|order_status|count|
+------------+-----+
|   delivered|96478|
|     shipped| 1107|
|    canceled|  625|
| unavailable|  609|
|    invoiced|  314|
|  processing|  301|
|     created|    5|
|    approved|    2|
+------------+-----+



In [14]:
delivery_df = orders_df.select('order_id', 'order_purchase_timestamp', 'order_delivered_customer_date')
delivery_df.show(5)

+--------------------+------------------------+-----------------------------+
|            order_id|order_purchase_timestamp|order_delivered_customer_date|
+--------------------+------------------------+-----------------------------+
|e481f51cbdc54678b...|     2017-10-02 10:56:33|          2017-10-10 21:25:13|
|53cdb2fc8bc7dce0b...|     2018-07-24 20:41:37|          2018-08-07 15:27:45|
|47770eb9100c2d0c4...|     2018-08-08 08:38:49|          2018-08-17 18:06:29|
|949d5b44dbf5de918...|     2017-11-18 19:28:06|          2017-12-02 00:28:42|
|ad21c59c0840e6cb8...|     2018-02-13 21:18:39|          2018-02-16 18:17:02|
+--------------------+------------------------+-----------------------------+
only showing top 5 rows



In [15]:
delivery_time = delivery_df.withColumn('delivery_time', datediff(col('order_delivered_customer_date'), col('order_purchase_timestamp')))
delivery_time.show(5)

+--------------------+------------------------+-----------------------------+-------------+
|            order_id|order_purchase_timestamp|order_delivered_customer_date|delivery_time|
+--------------------+------------------------+-----------------------------+-------------+
|e481f51cbdc54678b...|     2017-10-02 10:56:33|          2017-10-10 21:25:13|            8|
|53cdb2fc8bc7dce0b...|     2018-07-24 20:41:37|          2018-08-07 15:27:45|           14|
|47770eb9100c2d0c4...|     2018-08-08 08:38:49|          2018-08-17 18:06:29|            9|
|949d5b44dbf5de918...|     2017-11-18 19:28:06|          2017-12-02 00:28:42|           14|
|ad21c59c0840e6cb8...|     2018-02-13 21:18:39|          2018-02-16 18:17:02|            3|
+--------------------+------------------------+-----------------------------+-------------+
only showing top 5 rows



In [16]:
# Average Order Delivery Time

delivery_time.orderBy('delivery_time', ascending=False).show(5)

+--------------------+------------------------+-----------------------------+-------------+
|            order_id|order_purchase_timestamp|order_delivered_customer_date|delivery_time|
+--------------------+------------------------+-----------------------------+-------------+
|ca07593549f1816d2...|     2017-02-21 23:31:27|          2017-09-19 14:36:39|          210|
|1b3190b2dfa9d789e...|     2018-02-23 14:57:35|          2018-09-19 23:24:07|          208|
|440d0d17af552815d...|     2017-03-07 23:59:51|          2017-09-19 15:12:50|          196|
|2fb597c2f772eca01...|     2017-03-08 18:09:02|          2017-09-19 14:33:17|          195|
|285ab9426d6982034...|     2017-03-08 22:47:40|          2017-09-19 14:00:04|          195|
+--------------------+------------------------+-----------------------------+-------------+
only showing top 5 rows



In [17]:
order_payments_df.columns

['order_id',
 'payment_sequential',
 'payment_type',
 'payment_installments',
 'payment_value']

In [18]:
# Payment Distribution by Payment Method

order_payments_df.groupBy('payment_type').count().orderBy('count', ascending=False).show()

+------------+-----+
|payment_type|count|
+------------+-----+
| credit_card|76795|
|      boleto|19784|
|     voucher| 5775|
|  debit_card| 1529|
| not_defined|    3|
+------------+-----+



In [19]:
order_items_df.columns

['order_id',
 'order_item_id',
 'product_id',
 'seller_id',
 'shipping_limit_date',
 'price',
 'freight_value']

In [20]:
# Top products by sales amount

top_product = order_items_df.groupBy('product_id').agg(sum('price').alias('total_sales'), count('product_id').alias('count'))
top_product.orderBy('total_sales', ascending=False).show()

+--------------------+------------------+-----+
|          product_id|       total_sales|count|
+--------------------+------------------+-----+
|bb50f2e236e5eea01...|           63885.0|  195|
|6cdd53843498f9289...| 54730.20000000005|  156|
|d6160fb7873f18409...|48899.340000000004|   35|
|d1c427060a0f73f6b...| 47214.51000000006|  343|
|99a4788cb24856965...|43025.560000000085|  488|
|3dd2a17168ec895c7...| 41082.60000000005|  274|
|25c38557cf793876c...| 38907.32000000001|   38|
|5f504b3a1c75b73d6...|37733.899999999994|   63|
|53b36df67ebb7c415...| 37683.42000000001|  323|
|aca2eb7d00ea1a7b8...| 37608.90000000007|  527|
|e0d64dcfaa3b6db5c...|          31786.82|  194|
|d285360f29ac7fd97...|31623.809999999983|  123|
|7a10781637204d8d1...|           30467.5|  143|
|f1c7f353075ce59d8...|          29997.36|  154|
|f819f0c84a64f02d3...|29024.479999999996|   45|
|588531f8ec37e7d5f...|28291.989999999998|   20|
|422879e10f4668299...|26577.219999999972|  484|
|16c4e87b98a9370a9...|           25034.0

# Data Cleaning and Transformation

## **Identifing Missing Values**

In [21]:
def missing_values(df, df_name):
    print(f'Missing Values in: {df_name}')
    df.select([count(when(col(c).isNull(), 1)).alias(c) for c in df.columns]).show()

In [22]:
df_names = ['geolocation_df', 'products_df', 'sellers_df', 'translation_df', 'customer_df', 'order_items_df', 'order_payments_df', 'order_reviews_df', 'orders_df']
for name in df_names:
    missing_values(globals()[name], name)

Missing Values in: geolocation_df


+---------------------------+---------------+---------------+----------------+-----------------+
|geolocation_zip_code_prefix|geolocation_lat|geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+---------------+---------------+----------------+-----------------+
|                          0|              0|              0|               0|                0|
+---------------------------+---------------+---------------+----------------+-----------------+

Missing Values in: products_df
+----------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+----------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+------

## **Handling Missing Values**

1\. Drop missing values - non critical values

2\. Fill missing values - numerical values

3\. Impute missing values - for continous data

In [23]:
orders_df.columns

['order_id',
 'customer_id',
 'order_status',
 'order_purchase_timestamp',
 'order_approved_at',
 'order_delivered_carrier_date',
 'order_delivered_customer_date',
 'order_estimated_delivery_date']

**Dropping Missing Values**

In [24]:
order_df_cleaned = orders_df.na.drop(subset=['order_id', 'customer_id', 'order_status'])

In [25]:
missing_values(order_df_cleaned, 'Order Cleaned')

Missing Values in: Order Cleaned
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|order_id|customer_id|order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|       0|          0|           0|                       0|              160|                        1783|                         2965|                            0|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+



**Filling Missing Values**

In [26]:
order_df_cleaned = order_df_cleaned.fillna({'order_delivered_customer_date':'9999-12-31', 'order_approved_at':'9999-12-31', 'order_delivered_carrier_date':'9999-12-31'})

In [27]:
missing_values(order_df_cleaned, 'Orders Cleaned')

Missing Values in: Orders Cleaned
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|order_id|customer_id|order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|       0|          0|           0|                       0|                0|                           0|                            0|                            0|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+



**Imputing Missing Values**

In [28]:
from pyspark.ml.feature import Imputer

In [29]:
order_payments_df.columns

['order_id',
 'payment_sequential',
 'payment_type',
 'payment_installments',
 'payment_value']

In [30]:
imputer = Imputer(inputCols = ['payment_value'], outputCols=['payment_value_imputed']).setStrategy('mean')
payments_df_cleaned = imputer.fit(order_payments_df).transform(order_payments_df)

In [31]:
missing_values(order_payments_df, 'Order Payment')

Missing Values in: Order Payment
+--------+------------------+------------+--------------------+-------------+
|order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------+------------------+------------+--------------------+-------------+
|       0|                 0|           0|                   0|            0|
+--------+------------------+------------+--------------------+-------------+



## **Standardizing Data Types**

In [32]:
for name in df_names:
    print(f'\n Schema of {name}:')
    globals()[name].printSchema()


 Schema of geolocation_df:
root
 |-- geolocation_zip_code_prefix: integer (nullable = true)
 |-- geolocation_lat: double (nullable = true)
 |-- geolocation_lng: double (nullable = true)
 |-- geolocation_city: string (nullable = true)
 |-- geolocation_state: string (nullable = true)


 Schema of products_df:
root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (nullable = true)
 |-- product_height_cm: integer (nullable = true)
 |-- product_width_cm: integer (nullable = true)


 Schema of sellers_df:
root
 |-- seller_id: string (nullable = true)
 |-- seller_zip_code_prefix: integer (nullable = true)
 |-- seller_city: string (nullable = true)
 |-- seller_state: string (nullable = true)


 Schema of tran

In [33]:
customer_df_cleaned = customer_df.withColumn('customer_zip_code_prefix', col('customer_zip_code_prefix').cast('string'))

In [34]:
customer_df_cleaned.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



## **Remove Duplicate Records**

In [35]:
df_names

['geolocation_df',
 'products_df',
 'sellers_df',
 'translation_df',
 'customer_df',
 'order_items_df',
 'order_payments_df',
 'order_reviews_df',
 'orders_df']

In [36]:
orders_df.groupBy('order_id').count().filter('count>1').show()

+--------+-----+
|order_id|count|
+--------+-----+
+--------+-----+



In [37]:
order_df_cleaned = orders_df.dropDuplicates(['order_id'])

In [38]:
order_df_cleaned.groupBy('order_id').count().filter('count>1').show()

+--------+-----+
|order_id|count|
+--------+-----+
+--------+-----+



## **Normalizing Column Values**

In [39]:
payments_df_cleaned = payments_df_cleaned\
    .withColumn('payment_type', when(col('payment_type')=='boleto', 'Bank Transfer')\
                .otherwise(expr("concat_ws(' ', transform(split(payment_type, '_'), x -> initcap(x)))")))


In [40]:
payments_df_cleaned.show(10)

+--------------------+------------------+-------------+--------------------+-------------+---------------------+
|            order_id|payment_sequential| payment_type|payment_installments|payment_value|payment_value_imputed|
+--------------------+------------------+-------------+--------------------+-------------+---------------------+
|b81ef226f3fe1789b...|                 1|  Credit Card|                   8|        99.33|                99.33|
|a9810da82917af2d9...|                 1|  Credit Card|                   1|        24.39|                24.39|
|25e8ea4e93396b6fa...|                 1|  Credit Card|                   1|        65.71|                65.71|
|ba78997921bbcdc13...|                 1|  Credit Card|                   8|       107.78|               107.78|
|42fdf880ba16b47b5...|                 1|  Credit Card|                   2|       128.45|               128.45|
|298fcdf1f73eb413e...|                 1|  Credit Card|                   2|        96.12|      

## **Joining DataFrames**

In [43]:
order_with_details = order_df_cleaned.join(order_items_df, 'order_id', 'left').join(payments_df_cleaned, 'order_id', 'left').join(customer_df_cleaned, 'customer_id', 'left')
order_with_details.limit(4).toPandas()

,customer_id,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,freight_value,payment_sequential,payment_type,payment_installments,payment_value,payment_value_imputed,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,6489ae5e4333f3693df5ad4372dab6d3,000229ec398224ef6ca0657da4fc703e,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,1,c777355d18b72b67abbeef9df44fd0fd,...,17.87,1,Credit Card,5,216.87,216.87,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG
1,32e2e6ab09e778d99bf2e0ecd4898718,00054e8431b9d7675808bcb819fb4a32,delivered,2017-12-10 11:53:48,2017-12-10 12:10:31,2017-12-12 01:07:48,2017-12-18 22:03:38,2018-01-04,1,8d4f2bb7e93e6710a28f34fa83ee7d28,...,11.85,1,Credit Card,1,31.75,31.75,635d9ac1680f03288e72ada3a1035803,16700,guararapes,SP
2,9ed5e522dd9dd85b4af4a077526d8117,000576fe39319847cbb9d288c5617fa6,delivered,2018-07-04 12:08:27,2018-07-05 16:35:48,2018-07-05 12:15:00,2018-07-09 14:04:07,2018-07-25,1,557d850972a7d6f792fd18ae1400d9b6,...,70.75,1,Credit Card,10,880.75,880.75,fda4476abb6307ab3c415b7e6d026526,11702,praia grande,SP
3,16150771dfd4776261284213b89c304e,0005a1a1728c9d785b8e2b08b904576c,delivered,2018-03-19 18:40:33,2018-03-20 18:35:21,2018-03-28 00:37:42,2018-03-29 18:17:31,2018-03-29,1,310ae3c140ff94b03219ad0adc3c778f,...,11.65,1,Credit Card,3,157.60,157.60,639d23421f5517f69d0c3d6e6564cf0e,11075,santos,SP


## **Advanced Transformation or Feature Engineering**

In [45]:
order_items_df.select('price').summary().show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|            112650|
|   mean|120.65373901471354|
| stddev|183.63392805026012|
|    min|              0.85|
|    25%|              39.9|
|    50%|             74.99|
|    75%|             134.9|
|    max|            6735.0|
+-------+------------------+



In [44]:
quantiles = order_items_df.approxQuantile('price', [.01, .99], 0.0)
low_cutoff, high_cutoff = quantiles[0], quantiles[1]
low_cutoff, high_cutoff

(9.99, 890.0)

In [46]:
order_item_df_cleaned = order_items_df.filter((col('price') >= low_cutoff) & (col('price') <= high_cutoff))

In [47]:
order_item_df_cleaned.select('price').summary().show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|            110453|
|   mean|108.49213068006871|
| stddev|112.87303173792675|
|    min|              9.99|
|    25%|             39.99|
|    50%|              74.9|
|    75%|             130.0|
|    max|             890.0|
+-------+------------------+



In [50]:
products_df_cleaned = products_df.withColumn('product_size_category', when(col('product_weight_g') < 500, 'Small')\
                                             .when(col('product_weight_g').between(500, 2000), 'Medium').otherwise('Large'))

In [51]:
products_df_cleaned.select('product_weight_g', 'product_size_category').show()

+----------------+---------------------+
|product_weight_g|product_size_category|
+----------------+---------------------+
|             225|                Small|
|            1000|               Medium|
|             154|                Small|
|             371|                Small|
|             625|               Medium|
|             200|                Small|
|           18350|                Large|
|             900|               Medium|
|             400|                Small|
|             600|               Medium|
|            1100|               Medium|
|            7150|                Large|
|             250|                Small|
|             600|               Medium|
|             200|                Small|
|             800|               Medium|
|             400|                Small|
|             900|               Medium|
|            1700|               Medium|
|             500|               Medium|
+----------------+---------------------+
only showing top

## **Load the Processed Data**

In [52]:
!hadoop fs -mkdir /data/olist_processed_data

In [59]:
customer_df_cleaned.write.mode('overwrite').parquet('/data/olist_processed_data/customer_df_cleaned.parquet')

In [60]:
!hadoop fs -ls -h /data/olist_processed_data/

Found 1 items
drwxr-xr-x   - root hadoop          0 2025-09-01 19:37 /data/olist_processed_data/customer_df_cleaned.parquet


## **Hive Meta Data Store**

In [61]:
# CREATE EXTERNAL TABLE customer_df_cleaned(
#     customer_id STRING,
#     customer_unique_id STRING,
#     customer_zip_code_prefix STRING,
#     customer_city STRING,
#     customer_state STRING
# )
# STORED AS PARQUET
# LOCATION '/data/olist_processed_data/customer_df_cleaned.parquet';